In [13]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from collections import OrderedDict
import colorlover as cl
from IPython.display import HTML

In [2]:
WA = lambda x: np.average(x, axis=0, weights=df.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
def calculate_metrics(data):
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount'] / (data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount'] / data['vehicleValueAmount'])*100
    return data

In [3]:
a = OrderedDict([
    #('assetNumber',['count']),
    ('reportingPeriodActualEndBalanceAmount', ['mean']),
    ('pctDeal', ['sum']),
    #('originalLoanTerm',[WA]),
    #('remainingTermToMaturityNumber',[WA]),
    ('paymentToIncomePercentage',[WA]),
    #('LTV',[WA]),
    ('originalInterestRatePercentage',[WA]),
    #('currentDelinquencyStatus',['mean'])
    ])
colsKeyList = list(a.keys())

In [4]:
example = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
print(len(example.index))
example.head()

50


,code,state,category,total exports,beef,pork,poultry,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton
0,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
1,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
2,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
3,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
4,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95


In [5]:
#here in case we need to map state names to state codes
state_dict = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv',
                         usecols=['code','state'])
state_dict = state_dict.set_index('code')
state_dict = state_dict.to_dict()

In [6]:
D_2017_1_01 = 'https://raw.githubusercontent.com/Amcdonne95/SeniorDesignData/master/Drive%20Auto%20Receivables%20Trust/DRIVE%202017-1/2017/05%20May/2017.06.15_Drive%20Auto%20Receivables%20Trust%202017-1%20Data%20Tape.csv'

In [7]:
df = pd.read_csv(D_2017_1_01, memory_map=True, low_memory=False)
df = calculate_metrics(df)

In [8]:
df.head()

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,...,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount,pctDeal,LTV
0,SC,"=""6345520""",05-01-2017,05-31-2017,SC,10/2013,28203.21,72,11/2019,0.1932,...,-,-,0.0,0.0,-,0,False,-,0.000012,122.044268
1,SC,"=""6345701""",05-01-2017,05-31-2017,SC,11/2013,13786.00,72,05/2020,0.2499,...,-,-,0.0,0.0,-,0,False,-,0.000009,114.902484
2,SC,"=""6345822""",05-01-2017,05-31-2017,SC,11/2013,12515.00,60,08/2019,0.2100,...,-,-,0.0,0.0,-,0,False,-,0.000007,104.309052
3,SC,"=""6346124""",05-01-2017,05-31-2017,SC,10/2013,16699.00,60,11/2018,0.2100,...,-,-,0.0,0.0,-,0,False,-,0.000006,109.681445
4,SC,"=""1006458""",05-01-2017,05-31-2017,SC,05/2012,21404.16,72,08/2018,0.2499,...,-,-,0.0,0.0,-,0,True,0.00,0.000008,118.925214


In [9]:
stratTable = df[colsKeyList].groupby(by=[df['obligorGeographicLocation']]).agg(a)
stratTable = stratTable.reset_index()
stratTable.columns = ['stateCode','averageBalance', 'percentDeal', 'waPTI', 'waAPR']
stratTable['percentDeal'] = stratTable['percentDeal']*100
stratTable['waPTI'] = stratTable['waPTI']*100
stratTable['waAPR'] = stratTable['waAPR']*100
stratTable = stratTable.round(2)

In [10]:
#strat Table is multi-indexed columns
stratTable

,stateCode,averageBalance,percentDeal,waPTI,waAPR
0,AK,20328.95,0.12,12.35,19.59
1,AL,17072.44,3.01,12.04,20.59
2,AR,17036.03,1.89,11.90,18.55
3,AZ,16828.00,1.66,11.39,20.13
4,CA,17457.18,7.25,10.90,19.60
5,CO,17753.00,1.01,10.81,18.54
6,CT,16449.32,0.70,10.08,16.63
7,DC,15638.31,0.04,10.31,20.55
8,FL,16995.56,11.11,10.98,18.30
9,GA,17825.94,7.50,11.91,20.45


In [11]:
#columns to string is an issue rn
for col in stratTable.columns:
    stratTable[col] = stratTable[col].astype(str)

In [19]:
#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],
       #[0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

color = cl.scales['9']['seq']['OrRd']

stratTable['text'] = stratTable['stateCode'] + '<br>' + 'WA APR ' + stratTable['waAPR'] 

data = [ dict(
        type='choropleth',
        colorscale = color, 
        autocolorscale = False,
        locations = stratTable['stateCode'],
        z = stratTable['percentDeal'],#.astype(float),
        locationmode = 'USA-states',
        text = stratTable['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Loan Concentration")
        ) ]
layout = dict(
        title = 'Auto Loans<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='AutoLoanMap' )

In [15]:
color = cl.scales['9']['seq']['OrRd']
color

['rgb(255,247,236)',
 'rgb(254,232,200)',
 'rgb(253,212,158)',
 'rgb(253,187,132)',
 'rgb(252,141,89)',
 'rgb(239,101,72)',
 'rgb(215,48,31)',
 'rgb(179,0,0)',
 'rgb(127,0,0)']

In [16]:
HTML( cl.to_html(color) )

In [ ]:
def map_feature(data, stratDict):
    data = calculate_metrics(data)
    keyLis = list(stratDict.keys())
    #group all of the data by each state
    stratTable = data[keyLis].groupby(by=[data['obligorGeographicLocation']]).agg(stratDict).reset_index()
    stratTable.columns = ['stateCode','averageBalance', 'percentDeal', 'waPTI', 'waAPR']
    #Manipulate the view of the significant figures
    stratTable['percentDeal'] = stratTable['percentDeal']*100
    stratTable['waPTI'] = stratTable['waPTI']*100
    stratTable['waAPR'] = stratTable['waAPR']*100
    stratTable = stratTable.round(2)
    #make the whole DF strings for ease of view in the plotly map
    for col in stratTable.columns:
        stratTable[col] = stratTable[col].astype(str)
    
    return stratTable
    